In [17]:
from LandBase import *

def nts_to_db(con) :
    # 국세청(NTS)의 세무서 코드 정의를 DB에 저장
    
    # DataFrame 작성
    #df = pd.read_csv('./source_files/nts_officebasicinfo.csv', sep=',', dtype={'세무서코드':str}, encoding='utf-8')
    df = pd.read_csv('./source_files/nts_officebasicinfo.csv', sep=',', encoding='utf-8')
    df.rename(columns={'세무서코드':'nts_cd', '세무서명':'nts_nm'}, inplace=True)
    
    # DB처리
    table_name = 'nts_mst'
    sql = 'delete from ' + table_name
    con.execute(sql)
    df.to_sql(name=table_name, con=con, if_exists='append', index=False, chunksize=1000)

def nts_lawd_to_db(con, p_yyyymm) :
    # 국세청(NTS)의 세무서와 법정동 매핑 정보를 DB에 저장
    
    # DataFrame 작성
    df = pd.read_csv('./source_files/nts_lawd_mapping_201708.csv', sep=',', dtype={'세무서':str,'법정동코드':str}, encoding='utf-8')
    df = df[['세무서','법정동코드','사용유무']] # 필요한 컬럼만 지정
    df.rename(columns={'세무서':'nts_cd', '법정동코드':'lawd_cd', '사용유무':'use_yn'}, inplace=True)
    df['base_ym']=p_yyyymm # 컬럼 추가 (기준년월)
    
    # DB처리
    table_name = 'nts_lawd_map'
    sql = 'delete from ' + table_name
    con.execute(sql)
    df.to_sql(name=table_name, con=con, if_exists='append', index=False, chunksize=1000)

def tax_history_to_db(p_con, p_yyyy, p_ignore_flag=True) :
    # 근로소득세납부이력을 DB에 저장 (출처: 국가통계포털)
    table_name = 'tax_history'
    exist_flag = False

    # 기존 데이터를 무시하지 말라(p_ignore_flag==False)는 옵션이 주어지면
    # 같은 데이터가 이미 있는지 검사해서 있으면 함수를 종료함
    if (p_ignore_flag==False) :
        sql = 'select 1 as cnt from '+table_name +' where tax_yyyy = \'' + p_yyyy + '\' limit 1'
        exist_flag = check_data_exists(p_con, sql)
        if (exist_flag) :
            return
    
    # Open API 호출 (kosis.kr)
    url = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=' + kosis_kr_apikey \
    + '&format=json&jsonVD=Y' \
    + '&userStatsId=yyj1219/133/DT_13301_013/2/1/20170915101721&prdSe=Y' \
    + '&startPrdDe=' + p_yyyy \
    + '&endPrdDe=' + p_yyyy
    
    # json 형식으로 반환된 Open API 결과를 DataFrame으로 변환 (불필요한 항목은 빼고, 근로소득세만 남김)
    response = requests.get(url)
    df = pd.read_json(response.text)
    df.drop(['C1', 'C1_NM_ENG', 'C1_OBJ_NM', 'C1_OBJ_NM_ENG', 'C2', 'C2_NM_ENG', 'C2_OBJ_NM', 'C2_OBJ_NM_ENG', \
            'ITM_ID', 'ITM_NM', 'ITM_NM_ENG', 'ORG_ID', 'PRD_SE', \
            'TBL_ID', 'TBL_NM', 'UNIT_NM', 'UNIT_NM_ENG'], axis=1, inplace=True)
    #df = df[ (df['C1_NM']=='근로소득세') & (df['DT']!='-') ]
    df = df[ (df['DT']!='-') ]
    df.rename(columns={'PRD_DE':'tax_yyyy', 'C1_NM':'tax_type', 'C2_NM':'tax_office', 'DT':'tax_amount'}, inplace=True)

    # DB 처리
    con.execute('delete from ' + table_name + ' where tax_yyyy = \'' + p_yyyy + '\'')
    df.to_sql(name=table_name, con=p_con,  if_exists='append', index=False, chunksize=1000)
    con.execute('commit')


In [18]:
# DB Connection Open
con = get_db_connection()

# 국세청(NTS)의 세무서 코드 정의를 DB에 저장
#nts_to_db(con)

# 국세청(NTS)의 세무서와 법정동 매핑 정보를 DB에 저장
#nts_lawd_to_db(con, '201706')

# 근로소득세납부이력을 DB에 저장 (출처: 국가통계포털)
tax_history_to_db(con, '2016', p_ignore_flag=False)

# DB Connection Close
disconnect_db(con)

print('Program Finised')

Program Finised
